In [ ]:
import import_ipynb
import cnn_genre_classifier_spectrograms as mgr
import torch
import numpy as np
from pytorch_model_summary import summary


# path to file that stores spectrograms and genre labels for each processed segment
DATA_PATH = "/data/shared/GTZAN-DATASET/spectrograms_10segments.npy"

In [ ]:
# load and split processed dataset

X_train, X_validation, X_test, y_train, y_validation, y_test = mgr.load_data(DATA_PATH, test_size=0.10, validation_size=0.10)

In [ ]:
# load existing or create new model

model = mgr.new_model()
# model = mgr.load_model("./saved_models/model-1616675807trained.pth")

print(summary(model, torch.rand(1, 1, np.shape(X_train)[2], np.shape(X_train)[3]).to(torch.device("cuda:0"))), sep='\n')

In [ ]:
# training loop

history = model.fit(
    X_train, y_train,
    validation_data=(X_validation, y_validation), 
    epochs=40, 
    batch_size=128,
    log=True)

In [ ]:
# plot results

mgr.plot_history(history)

print("Max_train_acc:", max(history['acc']), "  Min_train_loss:", min(history['loss']))
print("Max_val_acc:", max(history['val_acc']), "  Min_val_loss:", min(history['val_loss']))
    
t_acc, t_loss = model.test(X_test, y_test, out=True)

In [ ]:
mgr.save_model(model, "./saved_models")